## Make driving data for single point runs by subsetting GSWP3

In [1]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                     
import shutil                                                                                                                                                       
import tempfile 
import random


/global/homes/j/jneedham/.conda/envs/myenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
site = 'bci'
lat = 9.154300000000
lon = -79.846100000000

if lon < 0 : 
     lon = 360 + lon
     print(lon)

280.1539


### Loop through the three streams (solar, precip and tphwl) and extract the lat lon of interest - save to NGEET directory 

In [4]:
yrs = np.arange(2013,2015,1).tolist()
months = np.arange(1,13,1).tolist()
print(months)
print(yrs)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[2014]


In [5]:
for yr in yrs  :
    print(yr)
    for  mon in months : 
        
        solar_full = '/global/cfs/cdirs/e3sm/inputdata/atm/datm7/atm_forcing.datm7.GSWP3.0.5d.v2.c180716/Solar3Hrly/clmforc.GSWP3.c2011.0.5x0.5.Solr.{}-{:02d}.nc'.format(yr, mon)
        solar_full = xr.open_dataset(solar_full,  decode_times=False)

        lon_mids = solar_full.LONGXY[0,0:720]
        lat_mids = solar_full.LATIXY[0:360,0]

        abslat = np.abs(lat_mids - lat)
        abslon = np.abs(lon_mids - lon)
        c = np.maximum(abslon, abslat)

        ([xloc],[yloc]) = np.where(c == np.min(c))

        lat_new = lat_mids[yloc]
        lon_new = lon_mids[xloc]

        solar = solar_full.isel(lon=[xloc],lat=[yloc])
        solar.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/solar/clmforc.GSWP3.c2011.0.5x0.5.Solr.{}.{}-{:02d}.nc'.format(site,site, yr, mon))


        # precip
        precip_full = '/global/cfs/cdirs/e3sm/inputdata/atm/datm7/atm_forcing.datm7.GSWP3.0.5d.v2.c180716/Precip3Hrly/clmforc.GSWP3.c2011.0.5x0.5.Prec.{}-{:02d}.nc'.format(yr, mon)
        precip_full = xr.open_dataset(precip_full,  decode_times=False)
        
        precip = precip_full.isel(lon=[xloc],lat=[yloc])
        precip.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/precip/clmforc.GSWP3.c2011.0.5x0.5.Prec.{}.{}-{:02d}.nc'.format(site,site, yr, mon))
        
        # tphwl
        tphwl_full = '/global/cfs/cdirs/e3sm/inputdata/atm/datm7/atm_forcing.datm7.GSWP3.0.5d.v2.c180716/TPHWL3Hrly/clmforc.GSWP3.c2011.0.5x0.5.TPQWL.{}-{:02d}.nc'.format(yr, mon)
        tphwl_full = xr.open_dataset(tphwl_full,  decode_times=False)
        
        tphwl = tphwl_full.isel(lon=[xloc],lat=[yloc])
        tphwl.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/tphwl/clmforc.GSWP3.c2011.0.5x0.5.TPQWL.{}.{}-{:02d}.nc'.format(site,site, yr, mon))
        


2014


## Combine them into a single  file  per year

In [6]:
for yr in yrs  :
    print(yr)
    for  mon in months :

        solar = '/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/solar/clmforc.GSWP3.c2011.0.5x0.5.Solr.{}.{}-{:02d}.nc'.format(site,site,yr,mon)
        solar = xr.open_dataset(solar,  decode_times=False)

        precip = '/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/precip/clmforc.GSWP3.c2011.0.5x0.5.Prec.{}.{}-{:02d}.nc'.format(site,site,yr,mon)
        precip = xr.open_dataset(precip,  decode_times=False)

        tphwl = '/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/tphwl/clmforc.GSWP3.c2011.0.5x0.5.TPQWL.{}.{}-{:02d}.nc'.format(site,site,yr,mon)
        tphwl = xr.open_dataset(tphwl,  decode_times=False)


        # the solar time is different than the other two, which messes up merge. Replace solar time with precip time
        solar['time'] = precip['time']

        combined = xr.merge([solar, precip, tphwl])

        combined.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/{}/CLM1PT_data/{}-{:02d}.nc'.format(site,yr,mon))


2014
